In [1]:
import os as os
import sys
from IPython.display import FileLink, display
import requests
import json
import math
from datetime import datetime
from fpdf import FPDF
import base64
import webbrowser
from pathlib import Path
import tempfile

In [2]:
%env PYTHONIOENCODING=utf-8


env: PYTHONIOENCODING=utf-8


In [ ]:
API_KEY = 'key'


In [22]:
GOOGLE_API_KEY = API_KEY  # Replace with your actual API key
RELEVANT_TOURIST_KEYWORDS = [
    "fort", "palace", "museum", "temple", "sanctuary", "park", "zoo", "viewpoint", "waterfall",
    "heritage", "monument", "lake", "beach", "cave", "garden", "national park", "wildlife", "trek", "hill",
    "valley", "cliff", "dam", "fountain", "art gallery", "historical", "observatory"
]
RELEVANT_CAFE_KEYWORDS = [
    "cafe", "coffee", "tea house", "bakery", "brewery", "patisserie", "espresso", "latte", "roastery", "bistro", 
    "dhaba", "bhojanalaya", "restaurant", "dining", "eatery", "snack bar", "food court", "pub", "bar", "tavern", 
    "canteen", "kitchen"
]

EATING_PLACE_TYPES = ["cafe", "restaurant", "bakery", "meal_takeaway", "meal_delivery", "bar", "food"]

RELEVANT_HOTEL_KEYWORDS = [
    "hotel", "resort", "motel", "inn", "lodge", "guest house", "homestay", "hostel",
    "suites", "retreat", "spa resort","dharamshala","ashram","rest house","dharmshala","lodging",
    "accommodation", "stay", "boarding house", "bed and breakfast", "holiday home",
    "vacation rental", "self-catering", "camping ground", "caravan park", "yurt camp"
]

trip_type_keywords = {
    "adventure": ["adventure", "bungee", "paragliding", "rafting", "trek", "zip line", "camping", "hiking", "skiing"],
    "relaxation": ["spa", "beach", "resort", "meditation", "wellness", "yoga", "hot spring"],
    "culture": ["museum", "heritage", "historical", "monument", "temple", "art gallery", "palace", "fort"],
    "food": ["restaurant", "cafe", "street food", "bakery", "wine", "brewery", "dining"],
    "nature": ["park", "garden", "waterfall", "lake", "sanctuary", "wildlife", "valley"],
    "shopping": ["mall", "market", "bazaar", "boutique", "souvenir", "shopping center"],
}

# Function to get coordinates from a place name
def get_coordinates(place_name):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": place_name, "key": GOOGLE_API_KEY}
    response = requests.get(url, params=params)
    data = response.json()
    if data["status"] == "OK":
        location = data["results"][0]["geometry"]["location"]
        return location["lat"], location["lng"]
    return None, None

def is_relevant_tourist_place(place):
    """Check if a place is relevant as a tourist attraction based on its name and types."""
    name = place.get("name", "").lower()
    types = place.get("types", [])

    EXCLUDED_TYPES = {
        "travel_agency", "tour_operator", "real_estate_agency",
        "car_rental", "insurance_agency", "lodging", "general_contractor"
    }

    INCLUDED_TYPES = {
        "tourist_attraction", "park", "museum", "art_gallery", "zoo", "amusement_park",
        "aquarium", "natural_feature", "place_of_worship", "hindu_temple", "church", "mosque",
        "synagogue", "historical_place", "campground"
    }

    # Check if any keyword exists in the name
    keyword_match = any(keyword in name for keyword in RELEVANT_TOURIST_KEYWORDS)

    # Check if it's explicitly a tourist type and not an excluded type
    type_match = any(t in types for t in INCLUDED_TYPES) and not any(t in types for t in EXCLUDED_TYPES)

    return keyword_match or type_match


# Function to check if a place is a relevant cafe or restaurant
def is_relevant_cafe(place):
    """Check if a place is relevant as a cafe or restaurant."""
    name = place.get("name", "").lower()
    types = place.get("types", [])
    return any(keyword in name for keyword in RELEVANT_CAFE_KEYWORDS) or "cafe" in types or "restaurant" in types

def deduplicate_places(places):
    seen = set()
    unique_places = []
    for place in places:
        pid = place.get("place_id") or place.get("name", "").lower()
        if pid not in seen:
            seen.add(pid)
            unique_places.append(place)
    return unique_places


# Function to check if a place is a relevant hotel/lodging
def is_relevant_hotel(place):
    """Check if a place is relevant as a hotel or lodging."""
    name = place.get("name", "").lower()
    types = place.get("types", [])
    return any(keyword in name for keyword in RELEVANT_HOTEL_KEYWORDS) or "lodging" in types or "hotel" in types


# Function to fetch places
def get_google_places(lat, lon, radius=30000, place_type="tourist_attraction"):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {"location": f"{lat},{lon}", "radius": radius, "type": place_type, "key": GOOGLE_API_KEY}
    response = requests.get(url, params=params)
    results = response.json().get("results", [])
    
    places = []
    for place in results:
        photo_url = None
        if "photos" in place:
            photo_reference = place["photos"][0]["photo_reference"]
            photo_url = get_photo_url(photo_reference)
            
         # Apply correct relevance function without removing any other functionality
        if (place_type == "tourist_attraction" and is_relevant_tourist_place(place)) or \
           (place_type == "cafe" and is_relevant_cafe(place)) or \
           (place_type == "lodging" and is_relevant_hotel(place)):
            places.append({
                "name": place.get("name"),
                "rating": float(place.get("rating", 0) or 0),
                "address": place.get("vicinity", "Unknown"),
                "types": place.get("types", []),
                "lat": place["geometry"]["location"]["lat"],
                "lon": place["geometry"]["location"]["lng"],
                "photo_url": photo_url,
                "price_level": place.get("price_level", "N/A"),

            })
          
    
    return filter_by_rating(sort_places(places, lat, lon))
    


# Function to calculate number of days between two dates
def calculate_days(start_date, end_date):
    from datetime import datetime
    start = datetime.strptime(start_date, "%d-%m-%Y")
    end = datetime.strptime(end_date, "%d-%m-%Y")
    return (end - start).days + 1  # Include the end date

# Function to fetch places based on trip type
def get_places_by_trip_type(lat, lon, trip_type):
    place_type_mapping = {
        "Adventure": ["amusement_park", "campground", "hiking_area"],
        "Relaxation": ["spa", "park", "natural_feature"],
        "History": ["museum", "historical_landmark"],
        "Wildlife": ["zoo", "aquarium", "national_park"],
        "Beaches": ["beach", "natural_feature"],
        "Urban Exploration": ["shopping_mall", "tourist_attraction", "art_gallery"]
    }
    selected_types = place_type_mapping.get(trip_type, ["tourist_attraction"])
    places = []
    for place_type in selected_types:
        url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {"location": f"{lat},{lon}", "radius": 50000, "type": place_type, "key": GOOGLE_API_KEY}
        response = requests.get(url, params=params)
        results = response.json().get("results", [])
        for place in results:
            photo_url = None
            if "photos" in place:
                photo_reference = place["photos"][0]["photo_reference"]
                photo_url = get_photo_url(photo_reference)
                price_level = place.get("price_level", "N/A")
            places.append({
                "name": place.get("name"),
                "rating": place.get("rating", "N/A"),
                "address": place.get("vicinity", "Unknown"),
                "types": place.get("types", []),
                "lat": place["geometry"]["location"]["lat"],
                "lon": place["geometry"]["location"]["lng"],
                "photo_url": photo_url,
                "price_level": price_level,
                "pricing_info": get_pricing_info(price_level)
            })
    return places

def filter_places_by_preference(places, trip_type):
    trip_keywords = {
        "Adventure": ['adventure', 'bungee', 'paragliding', 'rafting', 'trek', 'zip', 'camping', 'hiking', 'skiing', 'thrill', 'extreme','rock climbing', 'climbing'],
        "Relaxation": ['spa', 'resort', 'relaxation', 'retreat', 'yoga', 'garden', 'meditation'],
        "History": ['fort', 'museum', 'temple', 'heritage', 'monument', 'historical', 'palace', 'ruins'],
        "Wildlife": ['zoo', 'wildlife', 'bird sanctuary', 'national park', 'animal', 'safari'],
        "Beaches": ['beach', 'sea', 'coast', 'island', 'ocean', 'bay'],
        "Urban Exploration": ['mall', 'market', 'urban', 'downtown', 'shopping', 'tower', 'city']
    }

    keywords = trip_keywords.get(trip_type, [])
    filtered = []

    for place in places:
        name = place.get("name", "").lower()
        address = place.get("vicinity", "").lower()
        description = " ".join(place.get("types", [])).lower()
        combined_text = f"{name} {address} {description}"

        if any(keyword in combined_text for keyword in keywords):
            filtered.append(place)

    return filtered



# Function to estimate travel costs
def estimate_travel_cost(mode, distance):
    cost_per_km = {"car": 10, "bus": 5, "train": 8}  # Example cost per km
    return cost_per_km.get(mode, 0) * distance

# Function to calculate accommodation cost
def calculate_accommodation_cost(hotels, num_nights):
    avg_price_per_night = 3000  # Example price per night
    return avg_price_per_night * num_nights

# Function to calculate food expenses
def calculate_food_expenses(num_days, num_meals_per_day=3):
    avg_meal_cost = 300  # Example cost per meal
    return num_days * num_meals_per_day * avg_meal_cost

# Function to get photo URL
def get_photo_url(photo_reference, max_width=400):
    return f"https://maps.googleapis.com/maps/api/place/photo?maxwidth={max_width}&photo_reference={photo_reference}&key={GOOGLE_API_KEY}"

# Function to map price level to description
def get_pricing_info(price_level):
    price_mapping = {
        0: "Free",
        1: "Budget ($)",
        2: "Moderate ($$)",
        3: "Expensive ($$$)",
        4: "Luxury ($$$$)"
    }
    return price_mapping.get(price_level, "N/A")

# Function to print budget details
def print_budget_details(travel_cost, accommodation_cost, food_expenses):
    print("\n💰 **Estimated Budget:**")
    print(f"- Travel Cost: ₹{travel_cost}")
    print(f"- Accommodation Cost: ₹{accommodation_cost}")
    print(f"- Food Expenses: ₹{food_expenses}")
    print(f"**Total Estimated Cost: ₹{travel_cost + accommodation_cost + food_expenses}**")

# Functions to fetch hotels, cafes, activities, and transport facilities
def get_google_hotels(lat, lon, radius=25000):
    return get_google_places(lat, lon, radius, place_type="lodging")

def get_google_cafes(lat, lon, radius=30000):
    all_places = []
    for place_type in EATING_PLACE_TYPES:
        places = get_google_places(lat, lon, radius, place_type=place_type)
        all_places.extend(places)
    return deduplicate_places(all_places)


def get_google_activities(lat, lon, radius=30000):
    return get_google_places(lat, lon, radius, place_type="amusement_park")

def get_google_transport(lat, lon, radius=25000):
    return get_google_places(lat, lon, radius, place_type="taxi_stand")

# Function to calculate straight-line distance using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    """Compute the great-circle distance between two points using the Haversine formula."""
    if not all([lat1, lon1, lat2, lon2]):
        return float('inf')  # If any coordinate is missing, push to end

    R = 6371  # Earth's radius in km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return round(R * c, 2)  # Return distance rounded to 2 decimal places

# Helper function: Compute the average center of a list of places
def compute_center(places):
    if not places:
        return None, None
    avg_lat = sum(p['lat'] for p in places) / len(places)
    avg_lon = sum(p['lon'] for p in places) / len(places)
    return avg_lat, avg_lon

# Helper function: Optimize route using nearest neighbor heuristic
def optimize_route(places, start_lat, start_lon):
    if not places:
        return []
    unvisited = places.copy()
    route = []
    current_lat, current_lon = start_lat, start_lon
    while unvisited:
        next_place = min(unvisited, key=lambda p: calculate_distance(current_lat, current_lon, p['lat'], p['lon']))
        route.append(next_place)
        current_lat, current_lon = next_place['lat'], next_place['lon']
        unvisited.remove(next_place)
    return route

MIN_RATING = 3.0  # Set a minimum acceptable rating

def filter_by_rating(places):
    """Remove places with low ratings."""
    return [place for place in places if place.get("rating", 0) >= MIN_RATING]

def sort_places(places, center_lat, center_lon):
    """Sort places by rating (desc) and distance from center (asc)."""
    for place in places:
        if 'lat' in place and 'lon' in place:
            place['distance_from_center'] = calculate_distance(center_lat, center_lon, place['lat'], place['lon'])
        else:
            place['distance_from_center'] = float('inf')  # Push unknown distances to the end

    return sorted(places, key=lambda p: (-p.get('rating', 0), p['distance_from_center']))



def generate_eating_suggestions(day_places, cafes, num_suggestions=15):
    if not day_places or not cafes:
        return []
    
    # Extract valid locations only (filter out places without 'geometry')
    valid_day_places = [p for p in day_places if 'geometry' in p and 'location' in p['geometry']]
    
    if not valid_day_places:
        return []

    # Compute center coordinates
    day_center_lat = sum(p['geometry']['location']['lat'] for p in valid_day_places) / len(valid_day_places)
    day_center_lon = sum(p['geometry']['location']['lng'] for p in valid_day_places) / len(valid_day_places)

    valid_cafes = [c for c in cafes if 'geometry' in c and 'location' in c['geometry']]

    for cafe in valid_cafes:
        cafe_lat = cafe['geometry']['location']['lat']
        cafe_lon = cafe['geometry']['location']['lng']
        cafe['distance_from_day_center'] = calculate_distance(day_center_lat, day_center_lon, cafe_lat, cafe_lon)

    sorted_cafes = sorted(valid_cafes, key=lambda cafe: cafe['distance_from_day_center'])
    return sorted_cafes[:num_suggestions]


def filter_places_by_preference(places, trip_type):
    trip_keywords = {
        "Adventure": ['adventure', 'bungee', 'paragliding', 'rafting', 'trek', 'zip', 'camping', 'hiking', 'skiing'],
        "Relaxation": ['spa', 'resort', 'relaxation', 'retreat', 'yoga', 'garden', 'meditation'],
        "History": ['fort', 'museum', 'temple', 'heritage', 'monument', 'historical', 'palace', 'ruins'],
        "Wildlife": ['zoo', 'wildlife', 'bird sanctuary', 'national park', 'animal', 'safari'],
        "Beaches": ['beach', 'sea', 'coast', 'island', 'ocean', 'bay'],
        "Urban Exploration": ['mall', 'market', 'urban', 'downtown', 'shopping', 'tower', 'city']
    }

    keywords = trip_keywords.get(trip_type, [])
    filtered = []

    for place in places:
        name = place.get("name", "").lower()
        address = place.get("vicinity", "").lower()
        description = " ".join(place.get("types", [])).lower()
        combined_text = f"{name} {address} {description}"

        if any(keyword in combined_text for keyword in keywords):
            filtered.append(place)

    return filtered



# Updated itinerary function:
# Now takes only one argument "all_places" and internally computes the center,
# then optimizes the route and groups the places into days (4 stops per day).
def generate_itinerary(all_places, cafes=None):
    """Generate a structured itinerary by grouping places into days and suggesting nearby cafes."""
    if not all_places:
        return {}

    # Compute center of all places
    center_lat, center_lon = compute_center(all_places)

    # Optimize route starting from the computed center
    route = optimize_route(all_places, center_lat, center_lon)
    itinerary = {}

    day = 1
    for i in range(0, len(route), 4):  # 4 places per day
        day_places = route[i:i+4]

        # Compute center for the day's places
        day_center_lat, day_center_lon = compute_center(day_places)

        # Generate eating suggestions only if cafes are provided
        day_eating = generate_eating_suggestions(day_places, cafes) if cafes else []

        itinerary[f"Day {day}"] = {
            "places": [
                {
                    "name": p["name"],
                    "rating": p["rating"],
                    "distance_from_center": f"{calculate_distance(center_lat, center_lon, p['lat'], p['lon']):.2f} km"
                } for p in day_places
            ],
            "eating_places": [
                {
                    "name": cafe["name"],
                    "rating": cafe["rating"],
                    "distance_from_day_center": f"{calculate_distance(day_center_lat, day_center_lon, cafe['lat'], cafe['lon']):.2f} km"
                } for cafe in day_eating
            ]
        }

        day += 1

    return itinerary


# Function to get transport distances using Distance Matrix API
def get_transport_distances(start_lat, start_lon, end_lat, end_lon):
    transport_modes = ["driving", "walking", "bicycling", "transit"]
    results = {}
    for mode in transport_modes:
        url = "https://maps.googleapis.com/maps/api/distancematrix/json"
        params = {
            "origins": f"{start_lat},{start_lon}",
            "destinations": f"{end_lat},{end_lon}",
            "mode": mode,
            "key": GOOGLE_API_KEY
        }
        response = requests.get(url, params=params)
        data = response.json()
        if data["status"] == "OK" and data["rows"][0]["elements"][0]["status"] == "OK":
            distance = data["rows"][0]["elements"][0]["distance"]["text"]
            duration = data["rows"][0]["elements"][0]["duration"]["text"]
            results[mode] = (distance, duration)
        else:
            results[mode] = ("N/A", "N/A")
    return results

# Function to display places (prints to console)
def display_places(title, places):
    print(f"\n{title}:")
    if places:
        for place in places[:15]:
            print(f"- {place['name']} ({place['rating']} ⭐) - {place.get('address', 'Unknown')} (Distance from center: {place.get('distance_from_center', 'N/A')})")
            if place.get("photo_url"):
                print(f"  Photo: {place['photo_url']}")
    else:
        print("No results found.")
        

# Function to suggest all recommended places
def suggest_all_places(places, cafe_limit=20):
    """
    Displays grouped places (Tourist Places, Cafes, Hotels) with an extended list of cafes.
    Keeps original functionality but shows more eating options to the user.
    """
    grouped_places = {"Tourist Places": [], "Cafes": [], "Hotels": []}
    
    for place in places:
        if "lodging" in place["types"]:
            grouped_places["Hotels"].append(place)
        elif "cafe" in place["types"]:
            grouped_places["Cafes"].append(place)
        else:
            grouped_places["Tourist Places"].append(place)
    
    for category, cat_places in grouped_places.items():
        print(f"\n**{category}:**")

        # If category is "Cafes", sort and limit the results
        if category == "Cafes":
            # Sort by rating (desc), then distance (asc)
            sorted_cafes = sorted(cat_places, key=lambda x: (-x.get("rating", 0), x.get("distance_from_center", float("inf"))))
            for place in sorted_cafes[:cafe_limit]:  # Show top `cafe_limit` cafes
                name = place.get("name", "Unknown Cafe")
                rating = place.get("rating", "N/A")
                distance = place.get("distance_from_center", "Unknown")
                print(f"- {name} ({rating}⭐) - Distance: {distance}")
        else:
            for place in cat_places:
                name = place.get("name", "Unknown Place")
                rating = place.get("rating", "N/A")
                print(f"- {name} ({rating}⭐)")


# Function to print the trip details
def print_trip_details(start_location, final_destination, start_date, end_date, straight_line_distance, transport_distances):
    print(f"\n📍 **Trip Details:**")
    print(f"- Start Location: {start_location}")
    print(f"- Final Destination: {final_destination}")
    print(f"- Start Date: {start_date}")
    print(f"- End Date: {end_date}")
    print(f"\n**Distance from {start_location} to {final_destination}:**")
    print(f"- Straight-line distance: {straight_line_distance:.2f} km")
    for mode, (dist, time) in transport_distances.items():
        print(f"- {mode.capitalize()} distance: {dist} (Approx. {time})")

def generate_itinerary_3plus2_unique(tourist_places, cafes, days_limit=None):

    # Ensure sorting of tourist places
    itinerary = {}
    day_count = 1
    
    # Sets to track selected places and eateries
    selected_places = set()
    selected_eateries = set()
    
    # Divide tourist places into chunks of 3 per day
    for i in range(0, len(tourist_places), 3):
        if days_limit and day_count > days_limit:
            break
        
        # Select 3 unique attractions for the day
        day_places = []
        for place in tourist_places[i : i+3]:
            if place['name'] not in selected_places:
                day_places.append(place)
                selected_places.add(place['name'])
        
        # Ensure we have 3 places, if not, take the next available ones
        while len(day_places) < 3 and len(selected_places) < len(tourist_places):
            for place in tourist_places:
                if place['name'] not in selected_places:
                    day_places.append(place)
                    selected_places.add(place['name'])
                if len(day_places) == 3:
                    break
        
        # If no places left to add, break
        if not day_places:
            break

        # Compute average center for today's attractions
        avg_lat = sum(p['lat'] for p in day_places if p.get('lat')) / len(day_places)
        avg_lon = sum(p['lng'] for p in day_places if p.get('lng')) / len(day_places)

        # Sort cafes by proximity and filter unique ones
        sorted_cafes = sorted(
            cafes,
            key=lambda c: calculate_distance(avg_lat, avg_lon, c.get('lat'), c.get('lng')) if c.get('lat') and c.get('lng') else float('inf')
        )

        # Select 2 unique nearby eating places
        day_eating_places = []
        for eatery in sorted_cafes:
            if eatery['name'] not in selected_eateries:
                day_eating_places.append(eatery)
                selected_eateries.add(eatery['name'])
            if len(day_eating_places) == 2:
                break
        
        # If not enough eateries, try to add more unique ones
        while len(day_eating_places) < 2 and len(selected_eateries) < len(cafes):
            for eatery in cafes:
                if eatery['name'] not in selected_eateries:
                    day_eating_places.append(eatery)
                    selected_eateries.add(eatery['name'])
                if len(day_eating_places) == 2:
                    break

        # Store the day's itinerary
        itinerary[f"Day {day_count}"] = {
            "places": day_places,
            "eating_places": day_eating_places
        }
        
        day_count += 1

    return itinerary


def generate_itinerary_pdf(itinerary, trip_details):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", "B", 16)
    pdf.cell(0, 10, "Trip Itinerary", ln=True, align="C")
    pdf.ln(5)
    
    # Trip details
    pdf.set_font("Arial", size=12)
    pdf.cell(0, 10, f"Start: {trip_details['start_location']}", ln=True)
    pdf.cell(0, 10, f"Destination: {trip_details['final_destination']}", ln=True)
    pdf.cell(0, 10, f"Dates: {trip_details['start_date']} to {trip_details['end_date']}", ln=True)
    pdf.cell(0, 10, f"Straight-line distance: {trip_details['straight_line_distance']:.2f} km", ln=True)
    pdf.ln(10)

    for day, details in itinerary.items():
        pdf.set_font("Arial", "B", 14)
        pdf.cell(0, 10, day, ln=True)

        pdf.set_font("Arial", size=12)
        pdf.cell(0, 8, "Tourist Attractions:", ln=True)
        for activity in details['places']:
            name = activity.get('name', 'Unknown')
            rating = activity.get('rating', 'N/A')
            dist = activity.get('distance_from_center', 'N/A')
            pdf.multi_cell(0, 8, f"- {name} ({rating}*) - {dist} km")

        pdf.ln(5)

    filename = f"itinerary_{trip_details['final_destination'].replace(' ', '_')}.pdf"
    pdf.output(filename)
    return filename


In [23]:
# Main Execution
start_location = input("Enter start location: ")
final_destination = input("Enter the final destination: ")

start_date = input("Enter the start date (DD-MM-YYYY): ")
end_date = input("Enter the end date (DD-MM-YYYY): ")

# Calculate number of days between start and end date
trip_duration = calculate_days(start_date, end_date) 
print(f"\n📅 Your trip duration: {trip_duration} days.")

# Get user preference for trip type
trip_types = ["Adventure", "Relaxation", "History", "Wildlife", "Beaches", "Urban Exploration"]
trip_type = ""
while trip_type not in trip_types:
    trip_type = input("\nEnter the type of trip you want (Adventure, Relaxation, History, Wildlife, Beaches, Urban Exploration): ").strip().title()
    if trip_type not in trip_types:
        print("Invalid choice. Please select from the given options.")

# Get coordinates
start_lat, start_lon = get_coordinates(start_location)
final_lat, final_lon = get_coordinates(final_destination)

if start_lat and start_lon and final_lat and final_lon:
    # Fetch all places
    all_places = get_google_places(final_lat, final_lon)

    # Filter and sort tourist attractions only
    tourist_places = [place for place in all_places if is_relevant_tourist_place(place)]
    sorted_tourist_places = sort_places(filter_by_rating(tourist_places), final_lat, final_lon)

    # Fetch only relevant cafes and hotels
    final_cafes = filter_by_rating(get_google_cafes(final_lat, final_lon))
    final_hotels = get_google_hotels(final_lat, final_lon)

    # Add distance_from_center to each place manually
    for place in sorted_tourist_places:
        if 'geometry' in place and 'location' in place['geometry']:
            place['distance_from_center'] = calculate_distance(final_lat, final_lon, place['geometry']['location'].get('lat', 0), place['geometry']['location'].get('lng', 0))

    for cafe in final_cafes:
        if 'geometry' in cafe and 'location' in cafe['geometry']:
            cafe['distance_from_center'] = calculate_distance(final_lat, final_lon, cafe['geometry']['location'].get('lat', 0), cafe['geometry']['location'].get('lng', 0))

    for hotel in final_hotels:
        if 'geometry' in hotel and 'location' in hotel['geometry']:
            hotel['distance_from_center'] = calculate_distance(final_lat, final_lon, hotel['geometry']['location'].get('lat', 0), hotel['geometry']['location'].get('lng', 0))


    # Fetch places based on trip type
    trip_specific_places = get_places_by_trip_type(final_lat, final_lon, trip_type)

    # Generate itineraries (ONLY tourist attractions)
    full_tourist_itinerary = generate_itinerary_3plus2_unique(sorted_tourist_places, final_cafes, trip_duration)

    trip_specific_itinerary = generate_itinerary_3plus2_unique(trip_specific_places, final_cafes, trip_duration)


    # Limit itineraries based on trip duration
    limited_full_tourist_itinerary = {day: full_tourist_itinerary[day] for day in list(full_tourist_itinerary.keys())[:trip_duration]}
    limited_trip_specific_itinerary = {day: trip_specific_itinerary[day] for day in list(trip_specific_itinerary.keys())[:trip_duration]}

    # Distance calculations
    straight_line_distance = calculate_distance(start_lat, start_lon, final_lat, final_lon)
    transport_distances = get_transport_distances(start_lat, start_lon, final_lat, final_lon)

    # Print trip details
    print_trip_details(start_location, final_destination, start_date, end_date, straight_line_distance, transport_distances)

    # Show lists of tourist attractions, cafes, and hotels
    print("\n🌍 **List of All Tourist Attractions:**")
    for place in sorted_tourist_places:
        print(f"- {place['name']} ({place['rating']}⭐) - Distance: {place['distance_from_center']} km")

    print("\n☕ **List of Cafes:**")
    for cafe in final_cafes:
        print(f"- {cafe['name']} ({cafe['rating']}⭐) - Distance: {cafe['distance_from_center']} km")

    print("\n🏨 **List of Hotels:**")
    for hotel in final_hotels:
        print(f"- {hotel['name']} ({hotel['rating']}⭐) - Distance: {hotel['distance_from_center']} km")

    # ✅ **Print the new unique itineraries**
    print("\n🌍 **Itinerary for All Tourist Attractions:**")
    for day, details in full_tourist_itinerary.items():
        print(f"### {day}")
        print("Tourist Attractions:")
        for activity in details['places']:
            print(f"- {activity['name']} ({activity['rating']}⭐) - Distance: {activity['distance_from_center']} km")
        print("Nearby Eating Places:")
        for eatery in details['eating_places']:
            print(f"- {eatery['name']} ({eatery['rating']}⭐) - Distance: {eatery['distance_from_center']} km")
        print("\n")

    # Print the itinerary based on user preference
    # Log the filtered places based on user preference
    print(f"\n🎯 **Places filtered for trip type: {trip_type}**")
    if not trip_specific_places:
        print("No places found matching your trip type preference.")
    else:
        for place in trip_specific_places:
            name = place.get("name", "Unknown")
            rating = place.get("rating", "N/A")
            location = place.get("geometry", {}).get("location", {})
            distance = calculate_distance(final_lat, final_lon, location.get("lat", 0), location.get("lng", 0))
            print(f"- {name} ({rating}⭐) - Distance: {distance:.2f} km")


    print("\n🌟 **Itinerary Based on Your Preference:**")
    for day, details in trip_specific_itinerary.items():
        print(f"### {day}")
        print("Tourist Attractions:")
        for activity in details['places']:
            print(f"- {activity['name']} ({activity['rating']}⭐) - Distance: {activity.get('distance_from_center', 'N/A')} km")

        print("Nearby Eating Places:")
        for eatery in details['eating_places']:
            print(f"- {eatery['name']} ({eatery['rating']}⭐) - Distance: {eatery['distance_from_center']} km")
        print("\n")

    # Generate itinerary PDFs
    trip_details = {
        "start_location": start_location,
        "final_destination": final_destination,
        "start_date": start_date,
        "end_date": end_date,
        "straight_line_distance": straight_line_distance
    }

    itinerary_pdf_all = generate_itinerary_pdf(full_tourist_itinerary, trip_details)
    itinerary_pdf_preferred = generate_itinerary_pdf(trip_specific_itinerary, trip_details)

    # Estimate costs
    chosen_transport = "car"
    travel_distance = float(transport_distances.get(chosen_transport, (0, ""))[0])
    travel_cost = estimate_travel_cost(chosen_transport, travel_distance)
    accommodation_cost = calculate_accommodation_cost(final_hotels, trip_duration)
    food_expenses = calculate_food_expenses(trip_duration)

    # Print budget details
    print_budget_details(travel_cost, accommodation_cost, food_expenses)

    # Save to output file
    with open("output.txt", "w", encoding="utf-8") as f:
        f.write("🌍 **List of All Tourist Attractions:**\n")
        for place in sorted_tourist_places:
            f.write(f"- {place['name']} ({place['rating']}⭐) - Distance: {place['distance_from_center']} km\n")

        f.write("\n☕ **List of Cafes:**\n")
        for cafe in final_cafes:
            f.write(f"- {cafe['name']} ({cafe['rating']}⭐) - Distance: {cafe['distance_from_center']} km\n")

        f.write("\n🏨 **List of Hotels:**\n")
        for hotel in final_hotels:
            f.write(f"- {hotel['name']} ({hotel['rating']}⭐) - Distance: {hotel['distance_from_center']} km\n")

        f.write("\n🌍 **Itinerary for All Tourist Attractions:**\n")
        for day, details in full_tourist_itinerary.items():
            f.write(f"### {day}\n")
            for activity in details['places']:
                f.write(f"- {activity['name']} ({activity['rating']}⭐) - Distance: {activity['distance_from_center']} km\n")
            f.write("Nearby Eating Places:\n")
            for eatery in details['eating_places']:
                f.write(f"- {eatery['name']} ({eatery['rating']}⭐) - Distance: {eatery['distance_from_center']} km\n")
            f.write("\n")

        f.write("\n🌟 **Itinerary Based on Your Preference:**\n")
        for day, details in trip_specific_itinerary.items():
            f.write(f"### {day}\n")
            for activity in details['places']:
                f.write(f"- {activity['name']} ({activity['rating']}⭐) - Distance: {activity.get('distance_from_center', 'NA')} km\n")

            f.write("Nearby Eating Places:\n")
            for eatery in details['eating_places']:
                f.write(f"- {eatery['name']} ({eatery['rating']}⭐) - Distance: {eatery['distance_from_center']} km\n")
            f.write("\n")

    print("📂 Itinerary PDFs and text file generated successfully.")




📅 Your trip duration: 6 days.

📍 **Trip Details:**
- Start Location: delhi
- Final Destination: rishikesh
- Start Date: 20-02-2025
- End Date: 25-02-2025

**Distance from delhi to rishikesh:**
- Straight-line distance: 196.90 km
- Driving distance: 242 km (Approx. 4 hours 13 mins)
- Walking distance: 217 km (Approx. 2 days 1 hour)
- Bicycling distance: N/A (Approx. N/A)
- Transit distance: 205 km (Approx. 7 hours 13 mins)

🌍 **List of All Tourist Attractions:**
- Bhootnath Temple Rishikesh (4.7⭐) - Distance: 3.16 km
- Shree Neelkanth Mahadev Temple (4.7⭐) - Distance: 6.57 km
- Shri Maa Chandi Devi Temple, Haridwar (4.7⭐) - Distance: 22.61 km
- Parmarth Niketan Ashram (4.6⭐) - Distance: 2.64 km
- Maa Mansa Devi Temple, Haridwar (4.6⭐) - Distance: 21.02 km
- Saptrishi Ashram, Haridwar (4.5⭐) - Distance: 16.66 km
- Bharat Mata Temple, Haridwar (4.5⭐) - Distance: 17.1 km
- Shri Prem Nagar Ashram, Haridwar (4.5⭐) - Distance: 25.09 km
- Bhuma Niketan (4.4⭐) - Distance: 17.75 km
- Rajaji Nat